In [9]:
import pandas as pd
import os
import numpy
import datetime

In [10]:
#read claim sample file 
claims = pd.read_excel("C:\Users\AG61560\OneDrive - Anthem\Documents\Python Scripts/HH Claims 20200601.xlsx", sheet_name='Sheet1')

In [11]:
#input service month column
claims['Service Month'] = claims['DOS'] + pd.DateOffset(1) + pd.offsets.MonthBegin(-1)

In [12]:
#convert GRPNM to county + dual/non-dual
claims['County'] = claims['GRPNM'].map(lambda x: "Alameda" if "ALAMEDA" in x \
                                      else "Los Angeles" if "LOS ANGELES" in x \
                                      else "Sacramento" if "SACRAMENTO" in x \
                                      else "San Francisco" if "SAN FRANCISCO" in x \
                                      else "Santa Clara" if "SANTA CLARA" in x \
                                      else "Tulare" if "TULARE" in x \
                                      else "Other County")
claims['DualIndicator'] = claims['GRPNM'].map(lambda x: "Dual" if "/FULL DUAL" in x \
                                      else "Dual" if "/PARTDUAL" in x \
                                      else "Dual" if "/DUAL" in x \
                                      else "Non-Dual")

In [13]:
#sort by tin, mbrid, serivce month, then LINE_PD
claims.sort_values(by=['Claims_TIN','MBRID','HHStatus','Service Month','MOD1','LINE_PD'], inplace = True, ascending=(True,True,True,True,False,False))
claims = claims.drop_duplicates(subset=['CLM_NBR','LINE_NBR','MBRID'])

In [14]:
def CheckSubmission(claims):
    if claims['HHStatus'] in ["EN","EX"]:# \
#    and claims['HHEnrollDate'] <= claims['HHDisenrollDate']:
        if claims['HHEnrollDate'] <= claims['DOS']:# \
#        and claims['HHDisenrollDate'] >= claims['DOS']:
            if claims['BILLED'] > 0:
                if claims['LINE_PD'] > 0:
                    if claims['MOD1'] == "U1 " \
                    or claims['MOD1'] == "U2 " \
                    or claims['MOD1'] == "U3 " \
                    or claims['MOD1'] == "U4 " \
                    or claims['MOD1'] == "U5 " \
                    or claims['MOD1'] == "U6 ":
                        return 'Paid'
                    else:
                        return 'Claim should not be paid, modifier needs to be U1-U6 for payment'
                else:
                    if claims['MOD1'] == "U7 ":
                        return 'Outreach and Engagement attempt'
                    else:
                        return 'Claim not Paid'
            else:
                if claims['MOD1'] == "U7 ":
                    return 'Outreach and Engagement attempt'
                else:    
                    return 'Billed Amount needs to be greater than $0'
        else:
            if claims['MOD1'] == "U7 ":
                return 'Outreach and Engagement attempt'
            else:
                return 'Date of Service outside of HH Enrolled Dates'
#    elif claims['HHStatus'] in ["EN","EX"] \
#    and claims['HHEnrollDate'] > claims['HHDisenrollDate']:
#        return 'HHDisenrollDate is invalid'
    else:
        if claims['MOD1'] == "U7 ":
            if claims['LINE_PD'] > 0:
                return 'Claim should not be paid, modifier needs to be U1-U6 for payment'
            else:
                return 'Outreach and Engagement attempt'
        else:
            return 'Member Not Enrolled in Health Homes'

In [15]:
#apply the function
claims['CheckSubmission'] = claims.apply(CheckSubmission, axis=1)

In [16]:
#input if claim process date is one year more than DOS then do not pay

In [17]:
#define daterange indexing
def PMPMPeriod(claims):
    if claims['Service Month'] < pd.to_datetime('2019-07-01'):
        val = 1
    elif claims['Service Month'] < pd.to_datetime('2020-07-01'):
        val = 2        
    else:
        val = 3
    return val

In [18]:
#apply daterange index to dataframe
claims['PMPMPeriod'] = claims.apply(PMPMPeriod, axis = 1)

In [19]:
#create dataframe manually for rates check, if PMPM amount matches county rate table based on County, Dual Indicator, and Date of Servuce
data = {'County':  ['San Francisco', 'San Francisco', 'Alameda', 'San Francisco', 'San Francisco', 'Alameda', 'Alameda', 'Sacramento', 'Sacramento', 'Santa Clara', 'Santa Clara', 'Tulare', 'Tulare', 'Los Angeles', 'Los Angeles', 'San Francisco', 'San Francisco', 'Alameda', 'Alameda', 'Sacramento', 'Sacramento', 'Santa Clara', 'Santa Clara', 'Tulare', 'Tulare', 'Los Angeles', 'Los Angeles'],
        'DualIndicator': ['Non-Dual', 'Dual', 'Non-Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual', 'Non-Dual', 'Dual'],
        'StartDate': ['2018-07-01', '2018-07-01', '2018-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2019-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01', '2020-07-01'],
        'EndDate': ['2019-06-30', '2019-06-30', '2019-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-06-30', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31', '2020-12-31'],
        'PMPMPeriod': ['1', '1', '1', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '2', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3', '3'],
        'Rate': ['558.38', '123.11', '449.08', '399.55', '91.73', '591.14', '142.28', '491.55', '111.02', '575.22', '131.21', '484.13', '106.81', '519.67', '122.15', '229.19', '68.76', '500.49', '109.90', '412.21', '83.56', '473.88', '98.02', '391.58', '74.72', '436.44', '93.21'],
        }
rates = pd.DataFrame (data, columns = ['County','DualIndicator','StartDate','EndDate','PMPMPeriod','Rate'])

In [20]:
#convert relevant columns to matching types and merge
rates['PMPMPeriod'] = rates['PMPMPeriod'].astype(int)
rates['Rate'] = pd.to_numeric(rates['Rate'])
claims = pd.merge(claims, rates, on = ['County','DualIndicator','PMPMPeriod'])

In [21]:
#add column to show if overpayment or underpayment
def OverUnder(claims):
    if claims['LINE_PD'] == 0:
        return 'N'
    elif claims['LINE_PD'] > claims['Rate']:
        if claims['CheckSubmission'] == 'Paid':
            return 'Overpayment'
        else:
            return 'N'
    elif claims['LINE_PD'] < claims['Rate']:
        if claims['CheckSubmission'] == 'Paid':
            return 'Underpayment'
        else:
            return 'N'
    elif claims['LINE_PD'] == claims['Rate']:
        if claims['CheckSubmission'] == 'Paid':
            return 'Correct Amount'
        else:
            return 'N'
    else:
        return 'N'

In [22]:
#apply overunder function to dataframe
claims['OverUnder'] = claims.apply(OverUnder, axis = 1)

In [23]:
#Group all claims by MBRID and Service Month
sumclaims = claims.groupby(['MBRID','Service Month','County','DualIndicator','PMPMPeriod','Rate'])['LINE_PD'].sum().reset_index()
sumclaims

,MBRID,Service Month,County,DualIndicator,PMPMPeriod,Rate,LINE_PD
0,90001899D,2019-11-01,Tulare,Non-Dual,2,484.13,0.00
1,90003380D,2019-09-01,Tulare,Non-Dual,2,484.13,0.00
2,90003380D,2020-03-01,Tulare,Non-Dual,2,484.13,0.00
3,90003380D,2020-04-01,Tulare,Non-Dual,2,484.13,0.00
4,90005818F,2019-10-01,Tulare,Non-Dual,2,484.13,0.00
5,90005818F,2020-04-01,Tulare,Non-Dual,2,484.13,0.00
6,90011413C,2019-08-01,Sacramento,Non-Dual,2,491.55,0.00
7,90011413C,2019-10-01,Sacramento,Non-Dual,2,491.55,0.00
8,90011413C,2019-11-01,Sacramento,Non-Dual,2,491.55,0.00
9,90011413C,2019-12-01,Sacramento,Non-Dual,2,491.55,491.55


In [351]:
#determine if multiple payments were made per member per month
def PMPMDupe(sumclaims):
    if sumclaims['LINE_PD'] > sumclaims['Rate']:
        return 'Overpayment of PMPM'
    elif sumclaims['LINE_PD'] == 0:
        return 'N'
    else:
        return 'Single PMPM Payment Made'

In [352]:
#apply PMPMDupe function to sumclaims
sumclaims['PMPMDupe'] = sumclaims.apply(PMPMDupe, axis = 1)

In [353]:
#shrink sumclaims down to necessary columns
sumclaims = sumclaims[['MBRID','Service Month','PMPMDupe']]

In [354]:
#merge sumclaims back with claims to show PMPM aggregate
claims = pd.merge(claims, sumclaims, on = ['MBRID','Service Month'])

In [355]:
#define FinalCheck function
def FinalCheck(claims):
    if claims['CheckSubmission'] == 'Member Not Enrolled in Health Homes':
        if claims['LINE_PD'] > 0:
            return 'OVERPAID - MEMBER NOT HH ENROLLED. DENY CLAIM R51420.'
        else:
            return 'N'
    elif claims['CheckSubmission'] == 'Paid':
        if claims['OverUnder'] == 'Overpayment':
            return 'OVERPAID - CORRECT CASE RATE IS IN RATE COLUMN. RECOUP OVERAGE.'
        elif claims['OverUnder'] == 'Underpayment':
            return 'UNDERPAID - PAY REMAINDER OF CASE RATE SHOWN IN RATE COLUMN.'
        else:
            if claims['PMPMDupe'] == 'Overpayment of PMPM':
                return 'OVERPAID - MONTHLY CASE RATE WAS ALREADY PAID. PLEASE DENY WITH 2H TYPE CODE.'
            else:
                return 'N'
    elif claims['CheckSubmission'] == 'Claim not Paid':
        if claims['PMPMDupe'] == 'N':
            if claims['MOD1'] == 'U1 ':
                return 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.'
            elif claims['MOD1'] == 'U2 ':
                return 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.'            
            elif claims['MOD1'] == 'U3 ':
                return 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.'            
            elif claims['MOD1'] == 'U4 ':
                return 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.'            
            elif claims['MOD1'] == 'U5 ':
                return 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.'            
            elif claims['MOD1'] == 'U6 ':
                return 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.'            
            else:
                return 'N'
        else:
            return 'N'
    elif claims['CheckSubmission'] == 'Claim should not be paid, modifier needs to be U1-U6 for payment':
        return 'OVERPAID - INVALID MODIFIER. DENY CLAIM R51420.'
    elif claims['CheckSubmission'] == 'Outreach and Engagement attempt':
        return 'N'
    elif claims['CheckSubmission'] == 'Billed Amount needs to be greater than $0':
        return 'N'
    elif claims['CheckSubmission'] == 'Date of Service outside of HH Enrolled Dates':
        return 'N'
    elif claims['CheckSubmission'] == 'HHDisenrollDate is invalid':
        return 'N'
    else:
        return 'N'

In [356]:
#apply FinalCheck function to claims
claims['FinalCheck'] = claims.apply(FinalCheck, axis = 1)

In [357]:
#create shift columns
claims['MBRID_shift'] = claims['MBRID'].shift(1)
claims['Service Month_shift'] = claims['Service Month'].shift(1)
claims['CheckSubmission_shift'] = claims['CheckSubmission'].shift(1)

In [358]:
#def AnthemPayOrRecoup
def AnthemPayOrRecoup(claims):
    if claims['FinalCheck'] == 'OVERPAID - MEMBER NOT HH ENROLLED. DENY CLAIM R51420.':
        val = -claims['LINE_PD']
    elif claims['FinalCheck'] == 'OVERPAID - INVALID MODIFIER. DENY CLAIM R51420.':
        val = -claims['LINE_PD']
    elif claims['FinalCheck'] == 'UNDERPAID - PAY REMAINDER OF CASE RATE SHOWN IN RATE COLUMN.':
        val = claims['Rate'] - claims['LINE_PD']
    elif claims['FinalCheck'] == 'OVERPAID - CORRECT CASE RATE IS IN RATE COLUMN. RECOUP OVERAGE.':
        val = claims['Rate'] - claims['LINE_PD']
    elif claims['FinalCheck'] == 'OVERPAID - MONTHLY CASE RATE WAS ALREADY PAID. PLEASE DENY WITH 2H TYPE CODE.':
        if claims['MBRID'] == claims['MBRID_shift'] \
        and claims['Service Month'] == claims['Service Month_shift']:
            val = -claims['LINE_PD']
        else:
            val = 0
    elif claims['FinalCheck'] == 'UNDERPAID - PAY CASE RATE SHOWN IN RATE COLUMN.':
        if claims['MBRID'] == claims['MBRID_shift'] \
        and claims['Service Month'] == claims['Service Month_shift'] \
        and claims['CheckSubmission'] == 'Outreach and Engagement attempt':
            val = claims['Rate']
        elif claims['MBRID'] == claims['MBRID_shift'] \
        and claims['Service Month'] == claims['Service Month_shift'] \
        and claims['CheckSubmission'] != 'Outreach and Engagement attempt':
            val = 0
        else:
            val = claims['Rate']
    else:
        val = 0
    return val

In [359]:
#apply FinalCheck function to claims
claims['AnthemPayorRecoup'] = claims.apply(AnthemPayOrRecoup, axis = 1)

In [360]:
#Realign columns
claimsfinal = claims[['MBU','CLM_NBR','LINE_NBR','Claims_TIN','PRV_NAME','RENDER_PROV','RENDERPRVNM','MBRID','HHStatus','HHEnrollDate','HHDisenrollDate','GRPNM','POS','CPT','MOD1','DOS','BILLED','LINE_PD','CLM_PAID','FEE_SCHED','CLM_ACTION','ACTION_DESC','CLAIM_RECD_DATE','PROC_DT','POST_DT','Service Month','County','DualIndicator','Rate','CheckSubmission','OverUnder','PMPMDupe','FinalCheck','AnthemPayorRecoup']]

In [361]:
#Place raw data on one sheet, place payments on new sheet, place recoups on new sheet

In [362]:
#use macros to create a pivot (wishlist)

In [363]:
#Export to new excel
claimsfinal.to_excel(r"C:\Users\AG61560\OneDrive - Anthem\Documents\Python Scripts/HH Claims Analysis 20200601.xlsx", index = False)